In [3]:
import sys
import itertools
import logging
from math import sqrt
from operator import add
from os.path import join, isfile, dirname

from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession

import json

from pyspark.mllib.fpm import FPGrowth

In [5]:
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()
print("Running Spark Version %s" % (spark.version))

#### load business data ###
# path_business ="yizhan/Desktop/cs181/dataset/buiness.json"
path_business = "dataset/business.json"
df_business_raw = spark.read.json(path_business)
catDF = df_business_raw.select(df_business_raw["categories"])
catDF_iter = catDF.rdd.collect()                                                         

Running Spark Version 2.2.0


In [51]:
for item in catDF.rdd.first().categories:
    print (item)

Shopping
Shopping Centers


In [90]:
def toCSVLine(item):
    labels = item.categories
    print(labels) 
    isResturant = False
    for label in labels:
        if label == 'Restaurants' or label == 'Food' :
            isResturant = True                              
    if isResturant == True: 
        return [cat + ',' for cat in line if cat != 'Restaurants' and cat != 'Food']
    else:
        return ""
    

In [127]:
data = catDF.rdd.filter(lambda item : 'Restaurants' in item.categories or 'Food' in item.categories).map(lambda item: set([x for x in item.categories if x !='Restaurants' and x != 'Food'])).filter(lambda item: len(item) != 0)

In [128]:
count = 0
cat_iter = data.collect()
for each in cat_iter:    
    print(each)
    count += 1
    if count > 50:
        break

{'Convenience Stores', 'Soul Food'}
{'Coffee & Tea'}
{'Sandwiches'}
{'Florists', 'Grocery', 'Bakeries', 'Shopping', 'Flowers & Gifts'}
{'Sports Bars', 'Bars', 'Nightlife', 'Burgers', 'American (Traditional)'}
{'Chinese'}
{'Coffee & Tea', 'Bakeries'}
{'Breakfast & Brunch', 'Sandwiches', 'Mexican', 'Italian', 'Diners'}
{'Chicken Wings'}
{'Seafood', 'American (Traditional)'}
{'Burgers'}
{'Comfort Food', 'Bars', 'Nightlife', 'Canadian (New)', 'American (Traditional)'}
{'Coffee & Tea', 'Taiwanese'}
{'Bars', 'Nightlife', 'Burgers', 'Chicken Wings', 'American (Traditional)'}
{'Pubs', 'Nightlife', 'Irish', 'Bars'}
{'Chicken Wings', 'Pizza', 'Sandwiches'}
{'Bakeries'}
{'Grocery'}
{'Chinese'}
{'Nightlife', 'Bars'}
{'Pubs', 'Nightlife', 'Bars'}
{'Asian Fusion', 'Sushi Bars'}
{'Breakfast & Brunch', 'Delicatessen', 'French', 'Specialty Food'}
{'Bars', 'Nightlife', 'Wine & Spirits', 'Beer', 'Pubs', 'American (Traditional)'}
{'Mexican'}
{'Mexican', 'Fast Food', 'American (Traditional)'}
{'Delis', 'Gr

In [109]:
# L.saveAsTextFile('category_data.csv')

In [147]:
model = FPGrowth.train(data.map(lambda item : list(item)), minSupport=0.01, numPartitions=10)
result = model.freqItemsets().collect()

In [148]:
for fi in result:
    print(fi)

FreqItemset(items=['Ethnic Food'], freq=761)
FreqItemset(items=['Ethnic Food', 'Specialty Food'], freq=761)
FreqItemset(items=['Japanese'], freq=2186)
FreqItemset(items=['Juice Bars & Smoothies'], freq=1203)
FreqItemset(items=['Mexican'], freq=3913)
FreqItemset(items=['Mexican', 'Fast Food'], freq=783)
FreqItemset(items=['Mediterranean'], freq=1482)
FreqItemset(items=['Nightlife'], freq=7350)
FreqItemset(items=['Diners'], freq=1132)
FreqItemset(items=['American (New)'], freq=3802)
FreqItemset(items=['American (New)', 'American (Traditional)'], freq=864)
FreqItemset(items=['American (New)', 'Bars'], freq=1411)
FreqItemset(items=['American (New)', 'Bars', 'Nightlife'], freq=1411)
FreqItemset(items=['American (New)', 'Nightlife'], freq=1454)
FreqItemset(items=['Caterers'], freq=1479)
FreqItemset(items=['Caterers', 'Event Planning & Services'], freq=1479)
FreqItemset(items=['Bars'], freq=7014)
FreqItemset(items=['Bars', 'Nightlife'], freq=7014)
FreqItemset(items=['Ice Cream & Frozen Yogurt

In [ ]:
#  drop the single items set !!!!